Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6241fb2f39b788bff524a9cf8f2a350c3f492f2dc22b09b8917a5dc0c66e0846
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [35]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [36]:
data = [
("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)
      ]
schema = StructType([
                    StructField("date", StringType(), True),
                    StructField("category", StringType(), True),
                    StructField("quantity", IntegerType(), True),
                    StructField("revenue", IntegerType(), True)
                   ])
spark = SparkSession.builder.appName("Homework_03").getOrCreate()
spark_df = spark.createDataFrame(data, schema)
spark_df = spark_df.withColumn("date",F.to_date('date'))
spark_df.printSchema()
spark_df.show()

root
 |-- date: date (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- revenue: integer (nullable = true)

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [37]:
window_mean_revenue = Window().partitionBy('category')
spark_df = spark_df.withColumn('mean_revenue', F.mean('revenue').over(window_mean_revenue))
spark_df.show()

+----------+-----------+--------+-------+------------------+
|      date|   category|quantity|revenue|      mean_revenue|
+----------+-----------+--------+-------+------------------+
|2023-11-20|      Books|     150|   9000|10333.333333333334|
|2023-11-21|      Books|     200|  12000|10333.333333333334|
|2023-11-22|      Books|     180|  10000|10333.333333333334|
|2023-11-20|   Clothing|     300|  15000|           15000.0|
|2023-11-21|   Clothing|     280|  14000|           15000.0|
|2023-11-22|   Clothing|     320|  16000|           15000.0|
|2023-11-20|Electronics|     100|  12000|           12500.0|
|2023-11-21|Electronics|     110|  13000|           12500.0|
|2023-11-22|Electronics|     105|  12500|           12500.0|
+----------+-----------+--------+-------+------------------+



In [38]:
# Вариант 1
pivot_df_1 = spark_df.groupBy('category').pivot('date').mean('revenue')
pivot_df_1.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [39]:
# Вариант 2
pivot_df_2 = spark_df.groupBy('category').pivot('date').agg(F.round(F.mean('mean_revenue'), 2))
pivot_df_2.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|  10333.33|  10333.33|  10333.33|
|   Clothing|   15000.0|   15000.0|   15000.0|
|Electronics|   12500.0|   12500.0|   12500.0|
+-----------+----------+----------+----------+

